# 📚 Recommendations based on weighted ratings

With the limitations of calculating the average ratings you will now create recommendations based on the weightage average for individual books. Take a look at [Building a Recommendation System using weighted-average score](https://medium.com/@developeraritro/building-a-recommendation-system-using-weighted-hybrid-technique-75598b6be8ed) and implement this concept.

### 1. Calculate Weightage Average for Individual books average rating
Decide what value your mean vote acrosse the whole report (C) should be.



In [8]:
import pandas as pd
import numpy as np

In [16]:
#import datasets
ratings = pd.read_csv('/Users/ekaterinamazur/PycharmProjects/INFOMPPM1/Week 01/data/ratings.csv')
books = pd.read_csv('/Users/ekaterinamazur/PycharmProjects/INFOMPPM1/Week 01/data/books.csv')
users = pd.read_csv('/Users/ekaterinamazur/PycharmProjects/INFOMPPM1/Week 01/data/users.csv')


/var/folders/x6/923dwt55237392mlk00827nh0000gn/T/ipykernel_73849/692748840.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('/Users/ekaterinamazur/PycharmProjects/INFOMPPM1/Week 01/data/books.csv')


In [17]:
ratings.shape

(17431, 4)

In [20]:
#making all necessary values: total amount review
v= ratings.groupby('ISBN').agg({'Book-Rating':'count'}).sort_values(by='Book-Rating', ascending=False).reset_index().rename(columns={'Book-Rating': 'total_count'})

,ISBN,total_count
0,0316666343,212
1,0312195516,125
2,0446672211,104
3,0316601950,97
4,0452282152,90
...,...,...
1045,3423202947,1
1046,3423202327,1
1047,3379015180,1
1048,3257227809,1


In [49]:
#make merged df with main dataset review
merged_df = ratings.merge(v)

,Unnamed: 0,User-ID,ISBN,Book-Rating,total_count
0,1474,277427,0061009059,9,63
1,3382,278026,0061009059,8,63
2,25744,6251,0061009059,7,63
3,27407,6543,0061009059,8,63
4,33326,7915,0061009059,8,63
...,...,...,...,...,...
17426,777307,188022,0671016776,10,5
17427,859152,207782,0671016776,8,5
17428,863885,208829,0671016776,9,5
17429,939034,227447,0671016776,6,5


In [50]:
# making coolumn rating and merge again
R=ratings.groupby('ISBN').agg({'Book-Rating':np.average}).sort_values(by='Book-Rating', ascending=False).reset_index().rename(columns={'Book-Rating': 'rating'})
merged_df = merged_df.merge(R)
merged_df

,Unnamed: 0,User-ID,ISBN,Book-Rating,total_count,rating
0,1474,277427,0061009059,9,63,8.142857
1,3382,278026,0061009059,8,63,8.142857
2,25744,6251,0061009059,7,63,8.142857
3,27407,6543,0061009059,8,63,8.142857
4,33326,7915,0061009059,8,63,8.142857
...,...,...,...,...,...,...
17426,777307,188022,0671016776,10,5,7.600000
17427,859152,207782,0671016776,8,5,7.600000
17428,863885,208829,0671016776,9,5,7.600000
17429,939034,227447,0671016776,6,5,7.600000


In [52]:
C=merged_df['Book-Rating'].mean() # 7.88 #mean of rating
m=merged_df['Book-Rating'].quantile(0.70) # 17 # quantile

In [53]:
# final formula ((R*v)+ (C*m))/(v+m)
# Calculate all the components based on the above formula
merged_df['weight']=((merged_df.rating*merged_df.total_count)+ (C*m))/(merged_df.total_count+m)
merged_df

,Unnamed: 0,User-ID,ISBN,Book-Rating,total_count,rating,weight
0,1474,277427,0061009059,9,63,8.142857,8.115878
1,3382,278026,0061009059,8,63,8.142857,8.115878
2,25744,6251,0061009059,7,63,8.142857,8.115878
3,27407,6543,0061009059,8,63,8.142857,8.115878
4,33326,7915,0061009059,8,63,8.142857,8.115878
...,...,...,...,...,...,...,...
17426,777307,188022,0671016776,10,5,7.600000,7.810231
17427,859152,207782,0671016776,8,5,7.600000,7.810231
17428,863885,208829,0671016776,9,5,7.600000,7.810231
17429,939034,227447,0671016776,6,5,7.600000,7.810231


In [64]:
merged_df = merged_df.sort_values('weight', ascending=False).groupby(['ISBN','weight']).count()
merged_df = merged_df.reset_index()
merged_df #final dataset

,ISBN,weight,Unnamed: 0,User-ID,Book-Rating,total_count,rating
0,0020199600,8.130180,1,1,1,1,1
1,0020442203,8.241291,1,1,1,1,1
2,0028604199,7.912802,1,1,1,1,1
3,0060008032,7.494194,1,1,1,1,1
4,0060096195,8.080801,1,1,1,1,1
...,...,...,...,...,...,...,...
1045,3596259924,8.213022,1,1,1,1,1
1046,8408043641,7.634324,1,1,1,1,1
1047,8495618605,7.434324,1,1,1,1,1
1048,8806142100,7.940294,1,1,1,1,1


In [83]:
#make new dataset with necessary first 2 columns
df_new = merged_df.iloc [:, [0, 1]]
recommend_by_weight =  df_new.head(10)
recommend_by_weight

,ISBN,weight
0,0020199600,8.130180
1,0020442203,8.241291
2,0028604199,7.912802
3,0060008032,7.494194
4,0060096195,8.080801
5,0060173289,7.859355
6,0060175966,8.106511
7,0060194448,7.822883
8,006019491X,8.232315
9,0060256656,8.968726


In [84]:
recommend_by_weight

,ISBN,weight
0,0020199600,8.130180
1,0020442203,8.241291
2,0028604199,7.912802
3,0060008032,7.494194
4,0060096195,8.080801
5,0060173289,7.859355
6,0060175966,8.106511
7,0060194448,7.822883
8,006019491X,8.232315
9,0060256656,8.968726


### 2. Save the recommendations
Select the top 10 of the ratings based on weight and save the recommendations as `recommendations-ratings-weight.csv` and replace the file to the app directory. Make certain the file has the following columns: `ISBN;weight`

In [79]:
# code goes here
recommend_by_weight.to_csv('recommendations-ratings-weight.csv', sep=';', index=False)

### 3. Compare recommendations based on average rating and weighted ratings
Do you see big differences when you look at the interface?

All the recommendations looks different. And that's logical because we used different approaches to get top-10.